# Métodos de Selección de Features

La selección de features es un paso fundamental en el preprocesamiento de datos para modelos de Machine Learning. Consiste en identificar y seleccionar un subconjunto relevante de variables para usar en la construcción de modelos predictivos. La correcta selección de features mejora la precisión del modelo, reduce el sobreajuste, acelera el tiempo de entrenamiento y facilita la interpretación del modelo.

En lo que sigue te presento los llamados **Métodos de Wrapper**, los cuales evalúan la calidad de los features utilizando un modelo de Machine Learning específico. Estos suelen ser más precisos, pero también más costosos computacionalmente:

- **Eliminación Recursiva de Features (RFE)**
- **Eliminación Recursiva de Features con Validación Cruzada (RFECV)**
- **Selección Secuencial de Features (SFS)**

Veamos de qué se trata cada uno

# 1.- Eliminación Recursiva de Features (RFE)

Es una técnica utilizada para seleccionar las variables más importantes para un modelo. Su objetivo es eliminar las características menos relevantes, mejorando así el rendimiento del modelo y reduciendo su complejidad.

Generalmente, *RFE* se aplica a modelos que pueden proporcionar alguna medida de la importancia de las características como los modelos lineales o arboles de decisión. Entre los modelos donde se puede aplicar tenemos:

- Regresión Lineal.
- Regresión Logística.
- Arboles de decisión.
- Bosques aleatorios.
- Maquina de vectores de soporte
- Entre otros...

## Cómo funciona RFE?

   Los pasos para entender el funcionamiento del método *RFE* son los siguientes:
   
   1. **Entrenamiento inicial**: Entrena un modelo utilizando todas las características (Variables predictoras).
    
    
   2. **Evaluación de importancia**: Evalúa la importancia de cada característica medida de diferentes maneras dependiendo del modelo utilizado (coeficientes de la regresión lineal).
   
    
   3. **Eliminación de características**: Elimina las características menos importantes para el modelo.
   
   
   4. **Repetición de pasos**: Repite los pasos del 1 al 3 hasta alcanzar un número predefinido de características.

## Aplicación de RFE

Vamos a aplicar *RFE* sobre un conjunto de datos de prueba de 6 variables, que incluye la variable objetivo, para encontrar las 3 más importantes para un modelo de Rgresión Lineal.

In [4]:
# Librerias para generar datos de prueba
import numpy as np
import pandas as pd

# semilla
np.random.seed(0)

# generamos los datos aleatorios de 5 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100)
})


Asociamos linealmente a la variable objetivo, 3 de las 5 características: feature 1, 2 y 5. Esto nos asegura que el modelo sólo estará influenciado por estas tres características y entonces, al aplicar *RFE*, nos debe arrojar estas mismas en caso de funcionar correctamente.

In [5]:
# variable objetivo
y = 3*X['feature_2'] + 2*X['feature_5'] + X['feature_1'] + np.random.rand(100)

Ahora, juntamos los datos en un Dataframe para mejorar la visualización.

In [6]:
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['target'] = y
df.head()

feature_1  feature_2  feature_3  feature_4  feature_5    target
0   0.548814   0.677817   0.311796   0.906555   0.401260  3.695163
1   0.715189   0.270008   0.696343   0.774047   0.929291  3.756831
2   0.602763   0.735194   0.377752   0.333145   0.099615  3.532546
3   0.544883   0.962189   0.179604   0.081101   0.945302  6.072647
4   0.423655   0.248753   0.024679   0.407241   0.869489  3.242399

### 1.- Entrenamiento inicial
Entrenamos el modelo de regresión lineal con todas las características.

In [7]:
# libreria para usar la regresion lineal
from sklearn.linear_model import LinearRegression

# funcion para entrena el modelo
def train_model(X, y):
    model = LinearRegression()
    return model.fit(X,y)

In [8]:
# entrenamos el modelo
modelo = train_model(X, y)

### 2.- Evaluación de importancia
Para el caso de la regresión, debemos obtener los coeficientes de cada características una vez entrenado el modelo.

In [9]:
# funcion para obtener los coeficientes de cada caracteristica
def evaluate_model(model):
    coeficientes = model.coef_
    return coeficientes

In [10]:
# coeficiente de las caracteristicas
coefi = evaluate_model(modelo)
coefi

array([1.0169788 , 2.96313024, 0.01281724, 0.04742558, 1.94336239])

### 3.- Eliminación de características
Identificamos la característica con el coeficiente más pequeño (el valor absoluto) y la eliminamos.

In [11]:
# funcion que determina la caracteristica con menor valor de coeficiente y la elimina
def remove_feature(X, coeficientes):
    # indice del coficiente mas pequeño
    min_indice = np.argmin(np.abs(coeficientes))
    # nombre de la caracteristica correspondiente al indice anterior
    feature_remove = X.columns[min_indice]
    # eliminar caracteristica
    X = X.drop(columns=[feature_remove])
    # retornar matriz de caracteristica y nombre de la caracteristica eliminada
    return X, feature_remove
    

### 4.- Repetir hasta obtener el número deseado de características

Como queremos encontrar las 3 caracteristicas más relvantes para el modelo, repetimos el proceso anterior tres veces. Para automatizar el proceso, procedemos como sigue:

In [12]:
# numero de features a seleccionar
n_features = 3

# ejecutar proceso hasta llegar a 3 caracteristicas
while X.shape[1] > n_features:
    modelo = train_model(X,y)
    coefi = evaluate_model(modelo)
    X, feature_remove = remove_feature(X, coefi)
    
# mostrar las 3 caracteristicas mas importantes
print('Caracteristicas seleccionadas', X.columns)
    
    

Caracteristicas seleccionadas Index(['feature_1', 'feature_2', 'feature_5'], dtype='object')


> Los resultados confirman que los features 1, 2 y 5 son los mas importantea para el modelo de regresión lineal, tal como esperabamos. Y aquí redica la relevancia de este método y es que nos permite encontrar, de un conjunto grande de características, un número específico de ellas importantes para el modelo.

## Aplicación de RFE con Scikit-Learn

Ahora vamos a utilizar la librería Scikit Learn para aplicar este método y simplificar el proceso de eliminación recursiva. Trabajarémos sobre los mismos datos de prueba del ejemplo anterior para hacer una comparación de los resultados de ambos métodos. 

In [13]:
# features
X = df.drop(columns=['target'])
# variable objetivo
y = df['target']

Aplicamos RFE con la librería sklearn

In [14]:
# libreria para aplicar RFE
from sklearn.feature_selection import RFE

# crear modelo de regresion
modelo_2 = LinearRegression()

# Crear el objeto RFE y especificar el numero de caracteristicas deseadas
rfe = RFE(estimator=modelo_2, n_features_to_select=3)

# ajustar RFE a los datos
rfe.fit(X, y)

# obtener las caracteristicas seleccionadas por RFE
features_select = X.columns[rfe.support_]

# mostrar nombre
features_select

Index(['feature_1', 'feature_2', 'feature_5'], dtype='object')

> De esta forma, obtenemos las características más importantes para el modelo de regresión, que en este caso son los features 1, 2 y 5, coincidiendo con los resultados del ejemplo anterior.

# 2.- Eliminación Recursiva de Features con Validación Cruzada (RFECV)

Es una técnica de selección de características que mejora la metodología *RFE* al incoporar la validación cruzada para seleccionar automáticamente el número óptimo de características.

## Cómo funciona RFECV?

1. **Entrenamiento del modelo**: Similar a RFE, entrena el modelo usando todas las características.


2. **Validación cruzada**: Utiliza la validación cruzada para evaluar el rendimiento del modelo en diferentes subconjuntos de datos.


3. **Eliminación de características**: Elimina la característica menos importante y evalúa el modelo usando validación cruzada.


4. **Repetición de pasos**: Repite los pasos del 1 hasta el 3 hasta que se encuentre el conjunto óptimo de características que maximiza el rendimiento del modelo.

## Aplicación de RFECV con Scikit Learn

Vamos a utilizar un conjunto de datos de prueba de 11 características, incluyendo la variable objetivo, para aplicar RFECV y obtener las características mas importantes para un modelo de Regresión Lineal.  

In [15]:
# semilla
np.random.seed(0)

# 10 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100),
    'feature_6': np.random.rand(100),
    'feature_7': np.random.rand(100),
    'feature_8': np.random.rand(100),
    'feature_9': np.random.rand(100),
    'feature_10': np.random.rand(100)
})

Relacionamos la variable objetivo con 4 características: feature 1, 2, 5 y 7. Esto asegura que de las 10 caracteristicas, estas 4 influencian el modelo y por tanto, al aplicar RFECV, nos debe arrojar las mismas.

In [16]:
# variable objetivo
y = (
    3*X['feature_2'] + 
    2*X['feature_5'] + 
    X['feature_1'] + 
    X['feature_7']*X['feature_7'] +
    np.random.rand(100)
)
y.head()

0    4.008168
1    3.501436
2    3.946276
3    6.034817
4    3.321619
dtype: float64

Juntamos los datos en un Dataframe para visualizarlos mejor

In [17]:
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['target'] = y
df.head()

feature_1  feature_2  feature_3  feature_4  feature_5  feature_6  \
0   0.548814   0.677817   0.311796   0.906555   0.401260   0.310381   
1   0.715189   0.270008   0.696343   0.774047   0.929291   0.373035   
2   0.602763   0.735194   0.377752   0.333145   0.099615   0.524970   
3   0.544883   0.962189   0.179604   0.081101   0.945302   0.750595   
4   0.423655   0.248753   0.024679   0.407241   0.869489   0.333507   

   feature_7  feature_8  feature_9  feature_10    target  
0   0.174658   0.373216   0.039993    0.810839  4.008168  
1   0.327988   0.222864   0.639705    0.348192  3.501436  
2   0.680349   0.080532   0.408303    0.211455  3.946276  
3   0.063208   0.085311   0.377407    0.059383  6.034817  
4   0.607249   0.221396   0.809365    0.876027  3.321619

Dividimos los datos en entrenamiento y prueba

In [18]:
# libreria para dividir los datos
from sklearn.model_selection import train_test_split

# entrenamiento 70% y prueba 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Aplicamos RFECV sobre el modelo de regresión lineal indicando el paso, el número de pliegues de la validación cruzada y la métrica a utilizar para medir el rendimiento del modelo.

In [19]:
# importamos el metodo RFECV
from sklearn.feature_selection import RFECV

# creamos el modelo de regresion lineal
model_2 = LinearRegression()

# configuramos RFECV
rfecv = RFECV(
    estimator=model_2,
    step=1, # paso para eliminar caracteristicas
    cv=5, # numero de pliegues para la validacion cruzada
    scoring='neg_mean_squared_error' # uso del MSE negativo como metrica de rendimiento
)

# ajustar RFECV a los datos
rfecv.fit(X_train, y_train)

# mostrar numero optimo de caracteristicas
print('Numero optimo de caracteristicas: ', rfecv.n_features_)
# mostrar nombre de las caracteristicas
print('Caracteristicas seleccionadas: ', X_train.columns[rfecv.support_])

Numero optimo de caracteristicas:  4
Caracteristicas seleccionadas:  Index(['feature_1', 'feature_2', 'feature_5', 'feature_7'], dtype='object')


El RFECV arroja 4 variables óptimas para el modelo de Regresión Lineal: feature 1, 2, 5, y 7, mismas que han sido relacionadas al inicio con la variable objetivo y las esperadas como resultado.

RFECV recibe como parámetro el *scoring* que en este caso se asigna como el negativo del error cuadratico medio (MSE). Veamos una explicación del porqué:
> *Esto se debe a como funciona la API y la convención de algunas métricas de evaluación. El MSE es una métrica de error, lo que significa que debemos minimizar este valor para mejorar el rendimiento del modelo. Sim embargo, la mayoría de las funciones de Scikit learn están diseñadas para maximizar la métrica de evaluación. Por tanto, al usar MSE, Scikit Learn lo convierte en su valor negativo para que la función de maximización funcione correctamente. Al maximizar el negativo del MSE estamos minimizando el MSE real.*

# 3.- Selección Secuencial de Features (SFS)

Es una técnica de selección de características usada para reducir el número de las mismas en un modelo de manera más eficiente. Realiza la selección de manera secuencial, es decir, añade o elimina características una por una basandose en su impacto en el rendimiento del modelo.

## Cómo funciona SFS?

La implementación de SFS va a depender del tipo de selección que se realice: Forward Selection (Seleccion hacia adelante) o Backward Selection (Selección hacia atrás). Veámos el paso a paso de cada método:

### **Forward Selection**

1.- **Inicialización**: Comienza con un modelo vacío, sin variables predictoras.

2.- **Evaluación de características**: Evalúa todas las características individualmente y selecciona la que mejora el rendimiento del modelo.

3.- **Agregar caracerística**: agrega la característica seleccionada al modelo.

4.- **Repetir**: Repite el proceso, evaluando las características restantes en combinación con las ya seleccionadas hasta alcanzar un número máximo de características determinadas.

### **Backward Selection**

1.-**Inicialización**: Comienza con todas las caracteristicas disponibles.

2.- **Evaluación de características**: Evalúa todas las características individualmente y selecciona la que tiene el menor impacto negativo en el rendimiento del modelo.

3.- **Eliminar característica**: Elimina la característica seleccionada del modelo.

4.- **Repetir**: Repite el proceso hasta alcanzar un número de características determinadas.

En general, la selección hacia adelante y hacia atrás no arroja resultados equivalentes debido a la forma en que van apareciendo las variables. Además, una puede ser mucho más rápida que la otra según la cantidad solicitada de características: si tenemos 10 características y pedimos 7 características seleccionadas, la selección hacia adelante necesitaría realizar 7 iteraciones mientras que la selección hacia atrás solo necesitaría realizar 3.

## Diferencias entre SFS y RFE

- *SFS* ofrece tanto selección hacia adelante como hacia atrás, proporcinando mayor flexibilidad en la dirección del proceso de selección.

- *SFS* no requiere que el modelo exponga un atributo para mostrar las características seleccionadas.

- *SFS* puede ser más eficiente en ciertos casos ya que evalúa incrementos específicos en el rendimiento del modelo.

- *RFE* puede ser computacionalmente costoso debido a la necesidad de recalcular la importancia de las características y reentrenar el modelo en cada iteración.

- *SFS* puede ser más lento ya que debe evaluar más modelos.


## Aplicación de SFS con Scikit Learn

Vamos a utilizar el conjunto de datos de prueba del ejemplo anterior y vamos aplicar el método *SFS* tanto hacia adelante como hacia atrás.

In [20]:
# semilla
np.random.seed(0)

# 10 caracteristicas
X = pd.DataFrame({
    'feature_1': np.random.rand(100),
    'feature_2': np.random.rand(100),
    'feature_3': np.random.rand(100),
    'feature_4': np.random.rand(100),
    'feature_5': np.random.rand(100),
    'feature_6': np.random.rand(100),
    'feature_7': np.random.rand(100),
    'feature_8': np.random.rand(100),
    'feature_9': np.random.rand(100),
    'feature_10': np.random.rand(100)
})

Relacionamos la variable objetivo con 4 características: feature 1, 2, 5 y 7. Esto asegura que de las 10 caracteristicas, solo 4 influencian el modelo y por tanto, al aplicar *SFS* sólo algunas de estas, dependiendo de la cantidad que sean solicitadas, serán devueltas.

In [21]:
# variable objetivo
y = (
    3*X['feature_2'] + 
    2*X['feature_5'] + 
    X['feature_1'] + 
    X['feature_7']*X['feature_7'] +
    np.random.rand(100)
)
y.head()

0    4.008168
1    3.501436
2    3.946276
3    6.034817
4    3.321619
dtype: float64

Dividimos los datos entrenamiento y prueba

In [22]:
# libreria para dividir los datos
from sklearn.model_selection import train_test_split

# entrenamiento 70% y prueba 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Aplicamos SFS hacia adelante para un modelo de regresión lineal y solicitamos las 4 características más importantes.

In [26]:
# libreria para aplicar SFS
from sklearn.feature_selection import SequentialFeatureSelector

# Crear el modelo de regresión lineal
model_3 = LinearRegression()

# configurar el selector de caracteristicas sencuencial
sfs = SequentialFeatureSelector(
    model_3,
    n_features_to_select=4, #cantidad de caracteristicas
    direction='forward', # direccion de seleccion
    cv=5 # numero de validaciones cruzadas
    )

# ajustamos el selector a los datos de entrenamiento
sfs.fit(X_train, y_train)

# obtener las caracteristicas seleccionadas
selected_features = X_train.columns[sfs.get_support()]

# mostras caracteristicas
selected_features

Index(['feature_1', 'feature_2', 'feature_5', 'feature_7'], dtype='object')

> Vemos que las características seleccionadas son las mismas que están relacionadas con el modelo, por tanto, confirmamos que el método funciona correctamente.

Ahora, aplicamos SFS hacia atrás para un modelo de regresión lineal y una cantidad de 3 características. Esta elección de parámetros, hará que el procesos SFS demore un poco más debido a que deben realizar más iteraciones.

In [28]:
# Crear el modelo de regresión lineal
model_4 = LinearRegression()

# configurar el selector de caracteristicas sencuencial
sfs = SequentialFeatureSelector(
    model_4,
    n_features_to_select=3, #cantidad de caracteristicas
    direction='backward', # direccion de seleccion
    cv=5 # numero de validaciones cruzadas
                               )

# ajustamos el selector a los datos de entrenamiento
sfs.fit(X_train, y_train)

# obtener las caracteristicas seleccionadas
selected_features = X_train.columns[sfs.get_support()]

# mostras caracteristicas
selected_features

Index(['feature_2', 'feature_5', 'feature_7'], dtype='object')

> Las características seleccionadas por SFS hacia atrás están relacionadas con la variable objetivo del modelo, por tanto, confirmamos que el método funciona correctamente.

La selección de features es esencial para construir modelos de machine learning eficientes y precisos. La elección del método adecuado depende del tipo de datos, el modelo utilizado y el problema específico. Al aplicar técnicas de selección de features, es posible mejorar significativamente el rendimiento del modelo y obtener resultados más robustos y fáciles de interpreta